In [1]:
import gym
import pybullet
import pybullet_envs
from gym import wrappers
from datetime import datetime

# DR TRPO related files
from train_helper import *
from value import NNValueFunction
from utils import Logger
from dr_policy import DRPolicyKL, DRPolicyWass

In [4]:
#########################       Discrete State Space - KL DR Policy     ######################### 
# e.g. 'Taxi-v3', 'Roulette-v0', 'NChain-v0', 'FrozenLake-v0', 'CliffWalking-v0', 'FrozenLake8x8-v0'

env_name = 'Taxi-v3'
pybullet.connect(pybullet.DIRECT)
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyKL(sta_num, act_num)
val_func = NNValueFunction(1, 3)
gamma = 0.8
lam = 1
total_eps = 5000
batch_eps = 100
logger = Logger(logname=env_name + '_DR-KL_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        policy.update(observes, actions, advantages, disc_freqs)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 3, h2: 3, h3: 5, lr: 0.00577
***** Episode 100, Mean Return = -753.1, Mean Discounted Return = -19.0 *****
ExplainedVarNew: -4.72e-10
ExplainedVarOld: -1.52e-06
ValFuncLoss: 55.9


***** Episode 200, Mean Return = -592.6, Mean Discounted Return = -15.7 *****
ExplainedVarNew: -7.05e-10
ExplainedVarOld: -5.2e-10
ValFuncLoss: 50


***** Episode 300, Mean Return = -440.3, Mean Discounted Return = -12.4 *****
ExplainedVarNew: -0.000123
ExplainedVarOld: -9.07e-10
ValFuncLoss: 39.7


***** Episode 400, Mean Return = -347.7, Mean Discounted Return = -9.6 *****
ExplainedVarNew: -0.000548
ExplainedVarOld: -0.00032
ValFuncLoss: 26.7


***** Episode 500, Mean Return = -262.3, Mean Discounted Return = -8.0 *****
ExplainedVarNew: -0.00209
ExplainedVarOld: -0.00134
ValFuncLoss: 18.4


***** Episode 600, Mean Return = -196.5, Mean Discounted Return = -6.5 *****
ExplainedVarNew: -0.000354
ExplainedVarOld: -0.00248
ValFuncLoss: 11.7


***** Episode 700, Mean Return = -150.1, Mean Dis

In [5]:
#########################       Discrete State Space - Wasserstein DR Policy     ######################### 
# e.g. 'Taxi-v3', 'Roulette-v0', 'NChain-v0', 'FrozenLake-v0', 'CliffWalking-v0', 'FrozenLake8x8-v0'

env_name = 'Taxi-v3'
pybullet.connect(pybullet.DIRECT)
env = gym.make(env_name)
sta_num = env.observation_space.n
act_num = env.action_space.n
policy = DRPolicyWass(sta_num, act_num)
val_func = NNValueFunction(1, 3)
gamma = 0.8
lam = 1
total_eps = 5000
batch_eps = 100
logger = Logger(logname=env_name + '_DR-Wass_Batch=' + str(batch_eps), now=datetime.utcnow().strftime("%b-%d_%H:%M:%S"))


eps = 0
while eps < total_eps:
        trajectories = run_policy(env, policy, batch_eps, logger)
        eps += len(trajectories)
        # add estimated values to episodes
        add_value(trajectories, val_func)  
        # calculated discounted sum of Rs
        add_disc_sum_rew(trajectories, gamma, logger)  
        # calculate advantage
        add_gae(trajectories, gamma, lam)  
        # concatenate all episodes into single NumPy arrays
        observes, actions, advantages, disc_sum_rew = build_train_set(trajectories)
        disc_freqs = find_disc_freqs(trajectories, sta_num, gamma)
        log_batch_stats(observes, actions, advantages, disc_sum_rew, eps, logger)
        policy.update(observes, actions, advantages, disc_freqs)
        val_func.fit(observes, disc_sum_rew, logger)
        # write logger results to file and stdout
        logger.write(display=True) 
logger.close()

Value Params -- h1: 3, h2: 3, h3: 5, lr: 0.00577
***** Episode 100, Mean Return = -770.1, Mean Discounted Return = -19.7 *****
ExplainedVarNew: -1.43e-09
ExplainedVarOld: -3.26e-08
ValFuncLoss: 55.4


***** Episode 200, Mean Return = -720.8, Mean Discounted Return = -18.8 *****
ExplainedVarNew: -0.00142
ExplainedVarOld: -1.94e-09
ValFuncLoss: 59.2


***** Episode 300, Mean Return = -640.8, Mean Discounted Return = -15.9 *****
ExplainedVarNew: -0.00256
ExplainedVarOld: -0.00136
ValFuncLoss: 57.7


***** Episode 400, Mean Return = -569.9, Mean Discounted Return = -15.8 *****
ExplainedVarNew: -0.00117
ExplainedVarOld: -0.00123
ValFuncLoss: 60.2


***** Episode 500, Mean Return = -541.2, Mean Discounted Return = -15.8 *****
ExplainedVarNew: -0.000722
ExplainedVarOld: -0.00128
ValFuncLoss: 59.4


***** Episode 600, Mean Return = -417.0, Mean Discounted Return = -14.2 *****
ExplainedVarNew: -0.00403
ExplainedVarOld: -0.00152
ValFuncLoss: 63.2


***** Episode 700, Mean Return = -383.8, Mean D